In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dialogue2graph.datasets.complex_dialogues.generation import LoopedGraphGenerator

/home/peshkichev/miniconda3/envs/py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [3]:
from dialogue2graph.pipelines.core.graph import Graph
from dialogue2graph.pipelines.core.dialogue_sampling import RecursiveDialogueSampler
from dialogue2graph.datasets.complex_dialogues.prompts import graph_example_2

In [4]:
sampler = RecursiveDialogueSampler()

In [5]:
dias = sampler.invoke(Graph(graph_example_2),5)

ENDS:  [9]
NODES:  [[1, 2, 5, 10, 11, 6, 7, 9], [1, 5, 10, 11, 6, 7, 9], [1, 2, 5, 7, 9], [1, 3, 8, 9], [1, 4, 8, 9], [1, 5, 7, 9]]
REM:  [[1, 5, 10, 11, 6, 7, 9], [1, 2, 5, 7, 9], [1, 3, 8, 9], [1, 4, 8, 9]]
Graph has all the dialogues


In [9]:
type(dias[0])

dialogue2graph.pipelines.core.dialogue.Dialogue

In [6]:
from typing import Optional
from pydantic_settings import BaseSettings, SettingsConfigDict


class EnvSettings(BaseSettings, case_sensitive=True):

    model_config = SettingsConfigDict(env_file="./.env", env_file_encoding="utf-8")

    OPENAI_API_KEY: Optional[str]
    OPENAI_BASE_URL: Optional[str]
    HUGGINGFACE_TOKEN: Optional[str]
    EMBEDDER_DEVICE: Optional[str]

In [7]:
env_settings = EnvSettings()

In [5]:
from langchain.chat_models import ChatOpenAI

In [8]:
generation_model = ChatOpenAI(model="chatgpt-4o-latest", api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL, temperature=1)
validation_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL)

/tmp/ipykernel_3689305/2894222870.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  generation_model = ChatOpenAI(model="chatgpt-4o-latest", api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL, temperature=1)


In [9]:
generator = LoopedGraphGenerator(generation_model,validation_model)

INFO:dialogue2graph.utils.prompt_caching:Using in-memory cache


In [10]:
graph = generator.invoke("booking a table")

INFO:dialogue2graph.datasets.complex_dialogues.generation:Generating Graph ...



Generating graph for topic: booking a table


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dialogue2graph.datasets.complex_dialogues.generation:Graph generated is {'edges': [{'source': 1, 'target': 2, 'utterances': ["I'd like to book a table at a restaurant.", 'I want to reserve a table for dinner.']}, {'source': 2, 'target': 3, 'utterances': ['I have a place in mind.', 'Yes, I know where I want to go.']}, {'source': 2, 'target': 3, 'utterances': ['No, I need recommendations.', 'Can you suggest some options?']}, {'source': 3, 'target': 4, 'utterances': ['We’ll be 4 people.', 'Just 2 of us.']}, {'source': 4, 'target': 5, 'utterances': ['8 PM would be great.', 'Can we do 7:30 instead?']}, {'source': 5, 'target': 8, 'utterances': ["No, that's all. Thanks!", 'Nope, that was it. Appreciate it!']}, {'source': 8, 'target': 9, 'utterances': ['Thanks! Goodbye.', 'Alright, thanks again!']}, {'source': 5, 'target': 6, 'utterances': ['Actually, can I change the number

ENDS:  [9]
NODES:  [[1, 2, 3, 4, 5, 6, 3, 9], [1, 2, 3, 4, 5, 7, 4, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 3, 4, 9], [1, 2, 3, 9], [1, 2, 9]]
REM:  [[1, 2, 3, 4, 5, 6, 3, 9], [1, 2, 3, 4, 5, 7, 4, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 9]]


INFO:dialogue2graph.datasets.complex_dialogues.generation:Sampled 110 dialogues


Graph has all the dialogues
Graph has all the dialogues


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dialogue2graph.datasets.complex_dialogues.generation:Validating and fixing transitions...
INFO:dialogue2graph.datasets.complex_dialogues.generation:Validating initial graph
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx

Sampling dialogues...
ENDS:  [9]
NODES:  [[1, 2, 3, 4, 5, 6, 3, 9], [1, 2, 3, 4, 5, 7, 4, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 3, 4, 9], [1, 2, 3, 9], [1, 2, 9]]
REM:  [[1, 2, 3, 4, 5, 6, 3, 9], [1, 2, 3, 4, 5, 7, 4, 9], [1, 2, 3, 4, 5, 8, 9], [1, 2, 9]]


INFO:dialogue2graph.datasets.complex_dialogues.generation:going to return: {'edges': [{'source': 1, 'target': 2, 'utterances': ["I'd like to book a table at a restaurant.", 'I want to reserve a table for dinner.']}, {'source': 3, 'target': 4, 'utterances': ['We’ll be 4 people.', 'Just 2 of us.']}, {'source': 4, 'target': 5, 'utterances': ['8 PM would be great.', 'Can we do 7:30 instead?']}, {'source': 5, 'target': 8, 'utterances': ["No, that's all. Thanks!", 'Nope, that was it. Appreciate it!']}, {'source': 8, 'target': 9, 'utterances': ['Thanks! Goodbye.', 'Alright, thanks again!']}, {'source': 5, 'target': 6, 'utterances': ['Actually, can I change the number of people?', 'Wait, I need to adjust the party size.']}, {'source': 6, 'target': 3, 'utterances': ['Make it 5 instead.', 'I need to change it to 3 people.']}, {'source': 5, 'target': 7, 'utterances': ['Oops, I need to change the time.', 'Can we pick a different time instead?']}, {'source': 7, 'target': 4, 'utterances': ["Let's do

Graph has all the dialogues
Sampled 110 dialogues
Graph has all the dialogues


In [11]:
graph

[]